Table of Contents

1  1. Numpy Array to dataframe
1.1  1-1. Original Data Load
1.2  1-2. Array to DF
1.3  1-3 DF to Influx
1.3.1  Column Modify
1.3.2  Data Local Save : Classification
1.3.3  Original Data Upload
1.3.3.1  이 위에까지 돌려보시면 됩니다~
1.3.4  Data Split
2  2. dataFrame to Numpy Array
2.1  2-1. Influx Data Preparation
2.2  2-2. DF to Numpy Array

In [1]:
import sys

sys.path.append("../../..")
sys.path.append("../../../../")

1. Numpy Array to dataframe

1-1. Original Data Load

In [2]:
from Clust.clust.transformation.type.NPArrayToDF import trans3NPtoDF
import pickle
datafolderAddress = './ku_data/{}Test/'.format(problem)
train_x = pickle.load(open(datafolderAddress+'x_train.pkl', 'rb'))
train_y = pickle.load(open(datafolderAddress+'y_train.pkl', 'rb'))
test_x = pickle.load(open(datafolderAddress+'x_test.pkl', 'rb'))
test_y = pickle.load(open(datafolderAddress+'y_test.pkl', 'rb'))

In [27]:
print(train_x.shape) # 7352개의 액션, 9개의 Column 128길이의 시계열
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(7352, 9, 128)

In [ ]:
problem = "classification" # or regression

1-2. Array to DF

In [29]:
if problem == "regression":
    trainDF_X, trainDF_y = trans3NPtoDF(train_x, train_y, '2016-1-11')
    testDF_X, testDF_y = trans3NPtoDF(test_x, test_y, '2021-1-1')
elif problem == "classification":
    trainDF_X, trainDF_y = trans3NPtoDF(train_x, train_y, '1992-1-11')
    testDF_X, testDF_y = trans3NPtoDF(test_x, test_y, '2013-1-1')

In [38]:
trainDF_X

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8
1992-01-11 00:00:00,0.000181,0.010767,0.055561,0.030191,0.066014,0.022859,1.012817,-0.123217,0.102934
1992-01-11 00:11:15,0.010139,0.006579,0.055125,0.043711,0.042699,0.010316,1.022833,-0.126876,0.105687
1992-01-11 00:22:30,0.009276,0.008929,0.048405,0.035688,0.074850,0.013250,1.022028,-0.124004,0.102102
1992-01-11 00:33:45,0.005066,0.007489,0.049775,0.040402,0.057320,0.017751,1.017877,-0.124928,0.106553
1992-01-11 00:45:00,0.010810,0.006141,0.043013,0.047097,0.052343,0.002553,1.023680,-0.125767,0.102814
...,...,...,...,...,...,...,...,...,...
2012-02-26 23:03:45,0.022358,-0.280075,-0.180578,0.973228,1.083094,-0.226884,0.991497,-0.486260,-0.205803
2012-02-26 23:15:00,-0.024451,-0.248612,-0.153920,1.004266,1.187832,-0.313591,0.945067,-0.453405,-0.180733
2012-02-26 23:26:15,-0.071907,-0.194322,-0.127555,1.004855,1.156645,-0.362512,0.898095,-0.397775,-0.156105
2012-02-26 23:37:30,-0.142209,-0.147070,-0.092367,1.015589,1.100750,-0.383989,0.828372,-0.349247,-0.122798


1-3 DF to Influx

Column Modify

In [31]:
if problem == "regression":
    # columnName 수정
    rename_columns = {"col_0":"T1", "col_1":"RH_1", "col_2":"T2", "col_3":"RH_2", "col_4":"T3", "col_5":"RH_3", "col_6":"T4", "col_7":"RH_4",
    "col_8":"T5", "col_9":"RH_5", "col_10":"T6", "col_11":"RH_6", "col_12":"T7", "col_13":"RH_7", "col_14":"T8", "col_15":"RH_8",
    "col_16":"T9", "col_17":"RH_9", "col_18":"T_out", "col_19":"Press_mm_hg", "col_20":"RH_out", "col_21":"Windspeed", "col_22":"Visibility",
    "col_23":"Tdewpoint"}

    trainDF_X.rename(columns = rename_columns, inplace = True)
    testDF_X.rename(columns = rename_columns, inplace = True)

In [32]:
import pandas as pd
trainDF_y['value'] = pd.to_numeric(trainDF_y['value'],errors = 'coerce')
testDF_y['value'] = pd.to_numeric(testDF_y['value'],errors = 'coerce')

Data Local Save : Classification

In [14]:
if problem =="classification":
    trainDF_X.to_csv("classification_trainDF_X.csv")
    testDF_X.to_csv("classification_testDF_X.csv")
    trainDS_X = pd.read_csv("classification_trainDF_X.csv", chunksize=25000, index_col = "Unnamed: 0")
    testDS_X = pd.read_csv("classification_testDF_X.csv", chunksize=25000, index_col = "Unnamed: 0")

In [15]:
trainDS_X

In [16]:
testDS_X

Original Data Upload

In [17]:
from Clust.setting.influx_setting_KETI import CLUSTDataServer2 as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC

db_client = iC.InfluxClient(ins)

In [18]:
# 1. Original Upload
if problem =="regression":
     # Train X
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTrainX_10min"
    db_client.write_db(DBName, MSName, trainDF_X)
    
    # Train y
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTrainy_1day"
    db_client.write_db(DBName, MSName, trainDF_y)
    
    # Test X
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTestX_10min"
    db_client.write_db(DBName, MSName, testDF_X)
    
    # Test y
    DBName = "life_indoor_environment"
    MSName = "applianceEnergyDatasetTesty_1day"
    db_client.write_db(DBName, MSName, testDF_y)

elif problem =="classification":
    print("Start Classification Data Save")
    
    print("Start Data 1")
    # Test y 
    DBName = "life_action_pattern"
    MSName = "patternNameTest"
    db_client.write_db(DBName, MSName, testDF_y)
    
    print("Start Data 2")
    # Train y 
    DBName = "life_action_pattern"
    MSName = "patternNameTrain"
    db_client.write_db(DBName, MSName, trainDF_y)
    
    import time
    time.sleep(180)

Start Classification Data Save
Start Data 1
========== write success ==========
Start Data 2
========== write success ==========


In [19]:
if problem =="classification":
    print("Start Data 3")
    # Train X
    for train_X in trainDS_X:
        DBName = "bio_action_sensors"
        MSName = "accelerationTrain"
        db_client.write_db(DBName, MSName, train_X)
    
    print("Start Data 4")
    # Test X
    for test_X in testDS_X:
        DBName = "bio_action_sensors"
        MSName = "accelerationTest"
        db_client.write_db(DBName, MSName, test_X)

Start Data 3
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== write success ==========
========== writ

Data Split

In [20]:
if problem =="regression":
    trainDF_X_Temp = trainDF_X[["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9"]]
    trainDF_X_Humi = trainDF_X[["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9"]]
    trainDF_X_Out = trainDF_X[["T_out", "Press_mm_hg" ,"RH_out" ,"Windspeed" ,"Visibility" ,"Tdewpoint"]]

    testDF_X_Temp = testDF_X[["T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9"]]
    testDF_X_Humi = testDF_X[["RH_1", "RH_2", "RH_3", "RH_4", "RH_5", "RH_6", "RH_7", "RH_8", "RH_9"]]
    testDF_X_Out = testDF_X[["T_out", "Press_mm_hg" ,"RH_out" ,"Windspeed" ,"Visibility" ,"Tdewpoint"]]
    
    # 2. Split Data Upload
    # Train X
    DBName = "life_indoor_environment"
    MSName = "humidityTrain_10min"
    db_client.write_db(DBName, MSName, trainDF_X_Humi)
    
    DBName = "life_indoor_environment"
    MSName = "temperatureTrain_10min"
    db_client.write_db(DBName, MSName, trainDF_X_Temp)
    
    DBName = "weather_outdoor_environment"
    MSName = "belgiumChieverseAirportTrain_10min"
    db_client.write_db(DBName, MSName, trainDF_X_Out)
    
    # Test X
    DBName = "life_indoor_environment"
    MSName = "humidityTest_10min"
    db_client.write_db(DBName, MSName, testDF_X_Humi)
    
    DBName = "life_indoor_environment"
    MSName = "temperatureTest_10min"
    db_client.write_db(DBName, MSName, testDF_X_Temp)
    
    DBName = "weather_outdoor_environment"
    MSName = "belgiumChieverseAirportTest_10min"
    db_client.write_db(DBName, MSName, testDF_X_Out)

2. dataFrame to Numpy Array

2-1. Influx Data Preparation

In [22]:
# regression Original Data
# Train X
DBName = "life_indoor_environment"
MSName = "applianceEnergyDatasetTrainX_10min"
trainDF_X = db_client.get_data(DBName, MSName)

# Train y
DBName = "life_indoor_environment"
MSName = "applianceEnergyDatasetTrainy_1day"
trainDF_y = db_client.get_data(DBName, MSName)

In [23]:
trainDF_X.shape

(13680, 24)

2-2. DF to Numpy Array

In [24]:
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
dfX = trainDF_X 
dfy = trainDF_y
X, y = transDFtoNP(dfX, dfy)

/home/jwmoon/.conda/envs/sejong/lib/python3.8/site-packages/pandas/core/indexes/base.py:5278: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


In [25]:
X.shape

(95, 24, 144)

In [26]:
y.shape

(95,)